#### Lien vers le site

https://medium.com/jatana/unsupervised-text-summarization-using-sentence-embeddings-adb15ce83db1

# Premier summarizer

Je vais commencer par faire un premier type de summarizer, simple, basique, peu efficace, dans le but d'avoir une première base de travail. 

Ce dernier se décomposera en plusieurs étapes (7 précisément) et chacune des étapes fera office de partie au sein de ce notebook.

## Importation du'un modèle pour la langue francaise

In [1]:
import spacy
path = "C:\\Users\\Ivaros\\Documents\\Python\\NLP\\" #A modifier/automatiser
nlp = spacy.load(path + "fr_core_news_sm-2.2.5") #Charge le modele fr deja installe

## Lecture du texte

In [19]:
with open('C:\\Users\\Ivaros\\Documents\\Python\\Stage_3A\\Debat\\debat_2007.txt', 'r', encoding = "utf-8") as file:
     texte = file.read()       

## Tokenisation

Maintenant que j'ai récupéré le texte à résumer, je vais chercher à déterminer l'importance d'une phrase pour cela je commence par tokeniser mon texte phrase par phrase.

In [17]:
def return_token(sentence):
    """On va chercher a tokeniser la sentence"""
    doc = nlp(sentence) #On convertit la sentence en 'objet' spacy
    # Retourner le texte de chaque token
    return [X.text for X in doc]

def return_token_sent(texte):
    """Tokenisation du texte par phrase"""
    doc = nlp(texte)
    # Retourner le texte de chaque phrase
    return [X.text for X in doc.sents]

Tokenize = return_token_sent(texte)
Tokenize

['Arlette Chabot: Bonsoir à tous.',
 'Bonsoir',
 'Ségolène Royal, bonsoir, Nicolas Sarkozy.',
 "Vous êtes évidemment très nombreux, nous l'imaginons ce soir, pour suivre ce débat en France, mais aussi à l'étranger puisqu'il est diffusé en direct par de nombreuses chaînes en Europe, mais aussi dans le monde entier.",
 'Ce face à face est attendu.',
 "Il n'y en a pas eu depuis 1995, depuis 12 ans en France.",
 "Les Français vous ont donc choisi Ségolène Royal et Nicolas Sarkozy pour ce second tour de l'élection présidentielle.",
 "Nous sommes là avec  Patrick Poivre d'Arvor pour faire en sorte que ce débat se déroule dans la plus grande clarté.\n",
 "Patrick Poivre d'Arvor:",
 'Les règles seront les mêmes pour tout le monde, même temps de parole, mêmes questions.',
 'Les réponses, je le suppose, seront différentes.',
 "Avec Arlette Chabot, nous avons décidé de définir quatre grands chapitres de  durée égale : la conception du pouvoir et les institutions, les problèmes économiques et soci

On a bien une séparation par phrase excepté au début, avec le Bonsoir. 

## Vectorisation 

### Vectorisation mot par mot

In [4]:
import numpy as np

def return_word_embedding(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner le vecteur lié à chaque token
    return [(X.vector) for X in doc]

Pour la première phrase on constate 7 mots dont le point et les deux points. A supprimer ainsi que les stopwords par la suite.

### Vectorisation par phrase

On va "moyenner" les vecteurs d'une meme phrase pour pouvoir avoir une représentation vectorielle d'une phrase

In [6]:
def return_sentence_emb(sentence):
    """Prend en entree une phrase vectorise mot a mot"""
    somme = sentence[0]
    for i in range(len(sentence)) :
        if i != 0 :
            somme += sentence[i]
    return somme/len(sentence)

In [8]:
def return_text_emb(token_sent) : 
    """Vectorisation du texte"""
    text_vect = []    
    for i in range(len(token_sent)) :
        Sentence_emb = return_word_embedding(Tokenize[i]) #Vectorisation des phrases
        vect = return_sentence_emb(Sentence_emb)
        text_vect += [vect]
    return text_vect

## Clusterisation

Maintenant on va effectuer le Kmeans. Tout d'abord on va choisir le nombre de centre. Dans notre configuration on va avoir, un centre = une phrase. Comme on souhaite faire un résumé, on peut choisir de conserver X% du texte initial. Par exemple 10%. C'est ce nombre que l'on va conserver.

In [13]:
import numpy as np
from sklearn.cluster import KMeans

def clusterisation(texte,percent = 0.1) :
    """K_means sur le texte, percent correspond au pourcentage du texte que l'on veut garder"""
    Tokenize = return_token_sent(texte)
    nb_phrase = len(Tokenize)
    nb_centre = int(np.ceil(percent*nb_phrase)) #partie entiere avec np.ceil qui renvois un flottant -> int pour le kmeans
    emb_text = return_text_emb(Tokenize)
    kmeans = KMeans(n_clusters=nb_centre)
    kmeans = kmeans.fit(emb_text)
    return (kmeans, nb_centre)

In [28]:
#print(kmeans.labels_) #Pour avoir le numero du cluster attribue

In [11]:
from sklearn.metrics import pairwise_distances_argmin_min

def print_summary(texte) :
    """Renvoie le resume"""
    clust = clusterisation(texte)
    kmeans  = clust[0]
    nb_centre = clust[1]
    avg = []
    for j in range(nb_centre):
        idx = np.where(kmeans.labels_ == j)[0]
        avg.append(np.mean(idx))
    Tokenize = return_token_sent(texte)
    emb_text = return_text_emb(Tokenize)
    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, emb_text)
    ordering = sorted(range(nb_centre), key=lambda k: avg[k])
    summary = ' '.join([Tokenize[closest[idx]] for idx in ordering])
    return summary

In [ ]:
#Pour verifier que cela tourne
#summary = print_summary(texte)
#summary

Clairement à améliorer.

- Le decoupage/tokenisation n'est pas parfait et loin de la. 
- On peut supprimer la ponctuation au sein d'une phrase car elle n'apporte rien et rentre en compte par la suite dans l'embedding ? Mauvaise idée car la ponctuation peut finalement etre utile.
- On peut supprimer les stopwords qui n'apportent pas d'information ou peu.
- Ici on a affaire à une interview/debat. On peut donc localiser les entités nommées. (Utile pour le format réunion). 
- Localisation des différents sujets pour résumer par sujet. 
- La méthode d'embedding des phrases est brute et surement pas adaptée. Pensez à pondérer selon l'importance du mot ?
- Summarizer ? pas ouf....
- pk une dimension 32 dans l'embeding des mots
- ajouter les formules de politesse aux stopwords / les retirer dans tous les cas car n'apportent pas de sens
